# Simple Perceptron Model with Homomorphic Encryption


# Build Perceptron (Without Encryption)

In [83]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

## Define the Perceptron Model
We start by defining the Perceptron class, which includes initializing weights, forward pass, and weight update methods.

In [84]:
class Perceptron:
    def __init__(self, learning_rate=0.1, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)
                update = self.lr * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x > 0, 1, 0)


## Load Iris Dataset

In [85]:
iris = datasets.load_iris()
x = iris.data
y = iris.target


In [86]:
is_binary_class = (y == 0) | (y == 1)
x = x[is_binary_class]
y = y[is_binary_class]

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)

## Train the Perceptron

In [88]:
p = Perceptron(learning_rate=0.01, n_iters=100)
p.fit(x_train, y_train)


## Evaluate the Model
We can now test the trained model with the test dataset.

In [89]:
predictions = p.predict(x_test)
print("Model predictions:", predictions)
print("Actual labels:", y_test)


Model predictions: [0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0]
Actual labels: [0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0]


In [90]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%


# Build Perceptron (With Encryption)

In [91]:
pip install tenseal

In [92]:
# Generate a context for the CKKS scheme
import tenseal as ts
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

In [93]:
class EncryptedPerceptron:
    def __init__(self, context, learning_rate=0.1, n_iters=1000, n_features=None):
        self.context = context
        self.lr = learning_rate
        self.n_iters = n_iters
        self.n_features = n_features  # Number of features
        self.activation_func = self._unit_step_func
        self.weights = ts.ckks_vector(context, np.zeros(n_features))
        self.bias = ts.ckks_vector(context, [0])

    def fit(self, X, y):
        y_ = np.array([1 if i > 0 else 0 for i in y])

        for _ in range(self.n_iters):
            for idx, encrypted_x_i in enumerate(X):
                linear_output = encrypted_x_i.dot(self.weights) + self.bias
                y_predicted = self.activation_func(linear_output.decrypt())  # Decrypt on secure server

                update = self.lr * (y_[idx] - y_predicted)
                update_vector = update * np.array(encrypted_x_i.decrypt())  # Ensure the operation is on numpy array
                self.weights += ts.ckks_vector(self.context, update_vector)
                self.bias += ts.ckks_vector(self.context, [update])

    def predict(self, X):
        results = []
        for encrypted_x_i in X:
            linear_output = encrypted_x_i.dot(self.weights) + self.bias
            decrypted_output = linear_output.decrypt()  # Ensure decryption before the activation function
            y_predicted = self.activation_func(decrypted_output)
            results.append(y_predicted)
        return results

    def _unit_step_func(self, x):
        aggregate = sum(x)  # Sum up the decrypted list to get a single scalar value
        return 1 if aggregate > 0 else 0


In [98]:
x_train_encrypted = [ts.ckks_vector(context, x) for x in x_train]
x_test_encrypted = [ts.ckks_vector(context, x) for x in x_test]

In [99]:
p = EncryptedPerceptron(context, learning_rate=0.1, n_iters=100, n_features=4)
p.fit(x_train_encrypted, y_train)
#Takes about 5 minutes to train

In [100]:
predictions = p.predict(x_test_encrypted)
print("Model predictions:", predictions)
print("Actual labels:", y_test)

Model predictions: [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0]
Actual labels: [0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0]


In [101]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%
